# Child Mind Institute — Problematic Internet Use

## Team notes

from Célie, 11/2:

- Hi Anusha! I will label my work with a comment and my initials (CP).
- You can link the Kaggle notebook to your GitHub from within Kaggle.
- If using GitHub, save data locally in file kaggle_data (file is ignored by git)
- If using Kaggle notebook, data is located in: /kaggle/input/child-mind-institute-problematic-internet-use/
- I am using my local directory but if you are using Kaggle, you'll need to switch the commented out code.


In [1]:
# # CP: Show available files

# # CP: If using local directory
# # import os
# # for dirname, _, filenames in os.walk('kaggle_data'):
# #     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# # CP: If using Kaggle
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

####AG###

# CP
import numpy as np
import pandas as pd

In [3]:
test_data = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
column_names = list(test_data.columns)
train_data = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')

column_names = list(test_data.columns)

In [4]:
target = train_data['sii']
train_data = pd.DataFrame(train_data, columns = column_names)

train_data['sii'] = target

print(train_data.columns.difference(test_data.columns))
print(train_data.shape)
print(test_data.shape)

Index(['sii'], dtype='object')
(3960, 60)
(20, 59)


In [5]:
ids = test_data['id']

train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

In [6]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)
train_data, test_data = train_data.align(test_data, join='outer', axis = 1)

train_data.fillna(value=0, inplace=True)
test_data.fillna(value=0, inplace=True)

In [7]:
print(train_data.shape)
print(test_data.shape)

print(train_data.info())
print(test_data.info())

difference = (train_data.columns.difference(test_data.columns))
print(difference)

test_data = test_data.drop(columns=['sii'])  

(3960, 89)
(20, 89)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 89 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   BIA-BIA_Activity_Level_num              3960 non-null   float64
 1   BIA-BIA_BMC                             3960 non-null   float64
 2   BIA-BIA_BMI                             3960 non-null   float64
 3   BIA-BIA_BMR                             3960 non-null   float64
 4   BIA-BIA_DEE                             3960 non-null   float64
 5   BIA-BIA_ECW                             3960 non-null   float64
 6   BIA-BIA_FFM                             3960 non-null   float64
 7   BIA-BIA_FFMI                            3960 non-null   float64
 8   BIA-BIA_FMI                             3960 non-null   float64
 9   BIA-BIA_Fat                             3960 non-null   float64
 10  BIA-BIA_Frame_num                       

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X = train_data.drop(columns=['sii'])  
y = train_data['sii']  

#Scaling the training data. 
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

#Splitting the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

RFC = RandomForestClassifier(n_estimators=100, random_state=42)
RFC.fit(X_train, y_train)

y_pred_test = RFC.predict(X_test)
y_pred_train = RFC.predict(X_train)

print('Testing data: Model accuracy score with 100 decision-trees : {0:0.4f}'.format(accuracy_score(y_test, y_pred_test)*100))
print('Training data: Model accuracy score with 100 decision-trees : {0:0.4f}'.format(accuracy_score(y_train, y_pred_train)*100))

Testing data: Model accuracy score with 100 decision-trees : 74.4949
Training data: Model accuracy score with 100 decision-trees : 100.0000


In [9]:
# Now Logistic 
from sklearn.linear_model import LogisticRegression

X = train_data.drop(columns=['sii'])  
y = train_data['sii']  

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

LR = LogisticRegression()

LR.fit(X_train, y_train)

y_pred_test = LR.predict(X_test)
y_pred_train = RFC.predict(X_train)

print('Testing data accuracy: {0:0.4f}'.format(accuracy_score(y_test, y_pred_test)*100))
print('Training data accuracy: {0:0.4f}'.format(accuracy_score(y_train, y_pred_train)*100))

Testing data accuracy: 72.4747
Training data accuracy: 100.0000


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
# # RF on test data 
# X = test_data  
# X = scaler.fit_transform(X)
# y_pred = RFC.predict(X)

# submission = pd.DataFrame({
#    'id': ids,  
#    'sii': y_pred.astype(int) 
# })
# print(submission)

In [11]:
# LR Predicting on test data
X = test_data  
X = scaler.fit_transform(X)
y_pred = LR.predict(X)

#Creating the submission file
submission = pd.DataFrame({
    'id': ids,  
    'sii': y_pred.astype(int) 
})
print(submission)

          id  sii
0   00008ff9    0
1   000fd460    0
2   00105258    2
3   00115b9f    0
4   0016bb22    0
5   001f3379    0
6   0038ba98    2
7   0068a485    0
8   0069fbed    0
9   0083e397    0
10  0087dd65    0
11  00abe655    2
12  00ae59c9    3
13  00af6387    0
14  00bd4359    0
15  00c0cd71    1
16  00d56d4b    0
17  00d9913d    0
18  00e6167c    0
19  00ebc35d    0


In [12]:
submission.to_csv('submission.csv', index=False)